In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window



spark = SparkSession \
    .builder \
    .master("spark://spark-master:7077") \
    .appName("ejemplo_DF") \
    .getOrCreate()

df = spark.read\
 .option("header", "true")\
 .csv("hdfs://namenode:9000/input/data/bus_trips.csv")

# Filter for only regular services
df = df.filter(F.col("trip_type") == "regular")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 15:26:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.show()

+---------+--------+-----------+--------+---------+-----------+------------+------------+-----------------+----+-----+----+--------------------------+----+------+------------------------+-------------------+---------------------+-------------------+-----------------+
|trip_code| company|line_number|     bus|trip_type|     origin|origin_state| destination|destination_state|year|month| day|trip_start_hour_and_minute|hour|minute|trip_end_hour_and_minute|trip_duration_hours|travelled_distance_km|delay_start_minutes|delay_end_minutes|
+---------+--------+-----------+--------+---------+-----------+------------+------------+-----------------+----+-----+----+--------------------------+----+------+------------------------+-------------------+---------------------+-------------------+-----------------+
| 554e2d76|7980c0ae|   83ed87ea|3b3307d2|  regular|  TRES RIOS|          RJ|JUIZ DE FORA|               MG|2019| null|   1|                       528|   5|    28|                     652|         

In [18]:
df.groupby("trip_code").count().filter(F.col("count") > 1).count()

0

In [21]:
df.groupby("line_number").count().filter(F.col("count") > 0).count()

2327

In [4]:
from pyspark.sql.window import Window

w = Window.partitionBy("line_number")

# Convert necessary columns to numeric types
df = df.withColumn("trip_duration_hours", F.coalesce(F.col("trip_duration_hours"), F.avg("trip_duration_hours").over(w)).cast("float")) \
       .withColumn("delay_start_minutes", F.coalesce(F.col("delay_start_minutes"), F.lit(0)).cast("float")) \
       .withColumn("delay_end_minutes", F.coalesce(F.col("delay_end_minutes"), F.lit(0)).cast("float")) \
       .withColumn("travelled_distance_km", F.coalesce(F.col("travelled_distance_km"), F.avg("trip_duration_hours").over(w)).cast("float"))

# Calculate total delay, delay percentage, and average speed
df = df.withColumn("total_delay_minutes", F.col("delay_start_minutes") + F.col("delay_end_minutes")) \
       .withColumn("delay_percentage", (F.col("total_delay_minutes") / (F.col("trip_duration_hours") * 60)) * 100) \
       .withColumn("speed_kmh", F.round(F.col("travelled_distance_km") / F.col("trip_duration_hours"), 2))

# Fake the dates. If year 2020 we use month = 8 If year 2019, month = 7 and drop year
df = df.withColumn("month", F.when(F.col("year") == 2019, 7).otherwise(8))
df = df.withColumn("year", F.lit(2024))
df = df.withColumn("date", F.make_date(F.col("year"), F.col("month"), F.col("day")))
df = df.drop("trip_type", "year", "day")

# Save the intermediate table with partitioning by `day`
df.coalesce(1).write.mode("overwrite").partitionBy("date").bucketBy(4, "line_number").format("parquet").option("path", "hdfs://namenode:9000/output/bus_trips/intermediate").saveAsTable("intermediate")

In [5]:
# Create a standardized key for each line to combine two-way trips
df_lines = df.withColumn("line_key", F.when(F.col("origin") < F.col("destination"),
                                            F.concat_ws("-", F.col("origin"), F.col("destination")))
                                 .otherwise(F.concat_ws("-", F.col("destination"), F.col("origin"))))

# Aggregate data to keep only unique two-way records per line
lines_df = df_lines.select("company", "line_number", "line_key").distinct()

# Save this as a second intermediate table
lines_df.write.mode("overwrite").parquet("hdfs://namenode:9000/output/bus_trips/lines")

# Show results for verification
lines_df.show()

+--------+-----------+--------------------+
| company|line_number|            line_key|
+--------+-----------+--------------------+
|94179ee0|   3435ac8b|TEOFILO OTONI-VIT...|
|808be8bd|   7d1c7a7b|    FORTALEZA-RECIFE|
|95480e8f|   c2583349|    CAMPINAS-MARINGA|
|bf040c87|   9afa9729| CURITIBA-RIO DO SUL|
|68f76547|   148fbe87|BELO HORIZONTE-CO...|
|1c31a7b3|   9cf36911| CARANGOLA-ITAPERUNA|
|bf040c87|   57776c2b|   BLUMENAU-CURITIBA|
|1dffde3c|   98d99196|APARECIDA-VOLTA R...|
|1dffde3c|   e8acc322|RIO DE JANEIRO-VA...|
|d7951af4|   4ef44285|  FORTALEZA-TERESINA|
|94179ee0|   cb59a396|GOVERNADOR VALADA...|
|9caca56e|   01c699f3|    CURITIBA-ITARARE|
|cf535160|   86c5e497|    BOA VISTA-MANAUS|
|a4b92c82|   c0c0f650|  GOIANIA-UBERLANDIA|
|808be8bd|   2879bded|CAMPINA GRANDE-FO...|
|663c9c5c|   56fb9224|ANDRELANDIA-SAO P...|
|9bcede42|   fb2e26bd|   DOURADOS-LONDRINA|
|bf040c87|   a82134b3|FLORIANOPOLIS-SAO...|
|94179ee0|   3c150c3f|LINHARES-TEIXEIRA...|
|68f76547|   e8905f50|ITAPETINGA

In [6]:
lines_df.count()

2328

In [9]:
df = spark.read.parquet("hdfs://namenode:9000/output/bus_trips/intermediate")

# Define a window across all lines for comparison
window_spec = Window.partitionBy("line_number").orderBy("date").rowsBetween(Window.unboundedPreceding, 0)

# Groups all the trips within one date
df = df.groupby("date", "line_number").agg(
    F.avg("delay_percentage").alias("delay_percentage"),
    F.avg("trip_duration_hours").alias("trip_duration_hours"),
)
# Join overall statistics with trip summary and calculate ratios
trip_summary_ratios_df = df.withColumn("avg_delay_ratio", F.col("delay_percentage") / F.avg("delay_percentage").over(window_spec)) \
                           .withColumn("max_delay_ratio", F.col("delay_percentage") / F.max("delay_percentage").over(window_spec)) \
                           .withColumn("avg_trip_duration_hours", F.col("trip_duration_hours") / F.avg("trip_duration_hours").over(window_spec))\
                           .withColumn("max_trip_duration_hours", F.col("trip_duration_hours") / F.max("trip_duration_hours").over(window_spec))

trip_summary_ratios_df = trip_summary_ratios_df.drop("delay_percentage", "trip_duration_hours")

# Save results as the final output
trip_summary_ratios_df.write.partitionBy("date").mode("overwrite").parquet("hdfs://namenode:9000/output/bus_trips/stats")

In [7]:
trip_summary_ratios_df.count()

39506

In [10]:
trip_summary_ratios_df.show()

+----------+-----------+-------------------+--------------------+-----------------------+-----------------------+
|      date|line_number|    avg_delay_ratio|     max_delay_ratio|avg_trip_duration_hours|max_trip_duration_hours|
+----------+-----------+-------------------+--------------------+-----------------------+-----------------------+
|2024-07-02|   000cfbc6|                1.0|                 1.0|                    1.0|                    1.0|
|2024-07-03|   000cfbc6|-0.6884187248847855|-0.25606826738431093|     1.0434947756308663|                    1.0|
|2024-07-04|   000cfbc6| 1.5984731111606694|   0.848470964485128|     0.9600929982090063|     0.9020750071265196|
|2024-07-05|   000cfbc6|-1.2352197773714777| -0.3757181911446315|      0.864864873213882|     0.7775755516096283|
|2024-07-06|   000cfbc6| -2.968608153754544|-0.45326103068910006|     1.0487160653276284|     0.9544957793402483|
|2024-07-07|   000cfbc6| -5.601887082981342|-0.36861349143939864|     1.0747914992043215

In [3]:
spark.stop()